## Python code for the illustrative example in Longstaff & Schwartz's paper

In [3]:
import pandas as pd
import numpy as np

### Give condition for the option(rf, stock price and paths generated )

In [4]:
disc=0.94176

In [5]:
np.exp(-0.06)

0.9417645335842487

In [6]:
price_path=pd.DataFrame({'Path':np.arange(1,9,1), 't=0':np.ones(8), 't=1':[1.09,1.16,1.22,.93,1.11,.76,.92,.88],
                      't=2':[1.08,1.26,1.07,0.97,1.56,.77,.84,1.22],
                      't=3':[1.34,1.54,1.03,.92,1.52,.90,1.01,1.34]})

#build a table with title path t=0 t=1 t=2 t=3

In [7]:
price_path

,Path,t=0,t=1,t=2,t=3
0,1,1.0,1.09,1.08,1.34
1,2,1.0,1.16,1.26,1.54
2,3,1.0,1.22,1.07,1.03
3,4,1.0,0.93,0.97,0.92
4,5,1.0,1.11,1.56,1.52
5,6,1.0,0.76,0.77,0.90
6,7,1.0,0.92,0.84,1.01
7,8,1.0,0.88,1.22,1.34


In [8]:
CF=price_path.copy()
#return a new list, doesn't modify the original list 

In [9]:
CF['t=0']=CF['t=1']=CF['t=2']=0
# let t=0 t=1 t=2 all equals to 0 

In [10]:
CF['t=3']=np.maximum(1.1-CF['t=3'],0.0)
# Find payoff at t=3 (exercise or not )

In [11]:
CF 

,Path,t=0,t=1,t=2,t=3
0,1,0,0,0,0.00
1,2,0,0,0,0.00
2,3,0,0,0,0.07
3,4,0,0,0,0.18
4,5,0,0,0,0.00
5,6,0,0,0,0.20
6,7,0,0,0,0.09
7,8,0,0,0,0.00


In [12]:
price_path['ITM_t1']=np.where(price_path['t=1']<1.1,True,False)
price_path['ITM_t2']=np.where(price_path['t=2']<1.1,True,False)  # ITM = in-the-money
price_path['ITM_t3']=np.where(price_path['t=3']<1.1,True,False)
#price_path['ITM_t1'] adding another column to the table

### 找出 t1 t2 t3的stock price 是否 < exercise price
### 如果是，就True(exercise) 
### 如果不是，就False(不exercise)

In [13]:
price_path

,Path,t=0,t=1,t=2,t=3,ITM_t1,ITM_t2,ITM_t3
0,1,1.0,1.09,1.08,1.34,True,True,False
1,2,1.0,1.16,1.26,1.54,False,False,False
2,3,1.0,1.22,1.07,1.03,False,True,True
3,4,1.0,0.93,0.97,0.92,True,True,True
4,5,1.0,1.11,1.56,1.52,False,False,False
5,6,1.0,0.76,0.77,0.90,True,True,True
6,7,1.0,0.92,0.84,1.01,True,True,True
7,8,1.0,0.88,1.22,1.34,True,False,False


In [14]:
price_path.loc[price_path['ITM_t2']==True,'Y_reg_t2']=np.maximum(1.1-price_path.loc[price_path['ITM_t2'],'t=3'],0.0)*disc
#计算dicounted cashflow for t3 to t2并展现在table

In [15]:
price_path
# Y_reg_t2: dicount payoff of t3 to t2

,Path,t=0,t=1,t=2,t=3,ITM_t1,ITM_t2,ITM_t3,Y_reg_t2
0,1,1.0,1.09,1.08,1.34,True,True,False,0.000000
1,2,1.0,1.16,1.26,1.54,False,False,False,NaN
2,3,1.0,1.22,1.07,1.03,False,True,True,0.065923
3,4,1.0,0.93,0.97,0.92,True,True,True,0.169517
4,5,1.0,1.11,1.56,1.52,False,False,False,NaN
5,6,1.0,0.76,0.77,0.90,True,True,True,0.188352
6,7,1.0,0.92,0.84,1.01,True,True,True,0.084758
7,8,1.0,0.88,1.22,1.34,True,False,False,NaN


In [16]:
N_basis=2
x=price_path.loc[ price_path['ITM_t2'], 't=2' ].values
y=price_path.loc[ price_path['ITM_t2'], 'Y_reg_t2' ].values
reg = np.poly1d(np.polyfit(x,y,N_basis))  # Reminder: your will need this code for your final project!

$y = a_0 + a_1 x + a_2 x^{2} $

In [17]:
reg

poly1d([-1.81356745,  2.98339626, -1.0699825 ])

In [18]:
reg(1.5)

-0.6754148767124499

In [1]:
-1.0699825 + 2.98339626*1.5 - 1.81356745*(1.5**2)

-0.6754148724999993

Regression of Y (discounted value of t=3 payoff to t=2) against X (t=2 asset price) and $X^2$ gives us
   $E(Y|X) = -1.070 + 2.983X - 1.813X^{2}$

In [16]:
price_path['Cont(t=2)']=np.maximum(reg(price_path['t=2']),0.0)

In [17]:
price_path['Ex(t=2)']=np.maximum(1.1-price_path['t=2'],0)

In [18]:
#找出discounted 和 imeediately exercise 两个里面最大值
price_path['Ex(t=3)']=np.maximum(1.1-price_path['t=3'],0)
price_path['V(t=3)']=price_path['Ex(t=3)']
price_path['V(t=2)']=np.maximum(price_path['Ex(t=2)'],price_path['Cont(t=2)'])

In [19]:
price_path

,Path,t=0,t=1,t=2,t=3,ITM_t1,ITM_t2,ITM_t3,Y_reg_t2,Cont(t=2),Ex(t=2),Ex(t=3),V(t=3),V(t=2)
0,1,1.0,1.09,1.08,1.34,True,True,False,0.000000,0.036740,0.02,0.00,0.00,0.036740
1,2,1.0,1.16,1.26,1.54,False,False,False,NaN,0.000000,0.00,0.00,0.00,0.000000
2,3,1.0,1.22,1.07,1.03,False,True,True,0.065923,0.045898,0.03,0.07,0.07,0.045898
3,4,1.0,0.93,0.97,0.92,True,True,True,0.169517,0.117526,0.13,0.18,0.18,0.130000
4,5,1.0,1.11,1.56,1.52,False,False,False,NaN,0.000000,0.00,0.00,0.00,0.000000
5,6,1.0,0.76,0.77,0.90,True,True,True,0.188352,0.151968,0.33,0.20,0.20,0.330000
6,7,1.0,0.92,0.84,1.01,True,True,True,0.084758,0.156417,0.26,0.09,0.09,0.260000
7,8,1.0,0.88,1.22,1.34,True,False,False,NaN,0.000000,0.00,0.00,0.00,0.000000


In [20]:
price_path['CF_t=2']=0
price_path.loc[(price_path.ITM_t2) & (price_path['Cont(t=2)']<price_path['Ex(t=2)']),'CF_t=2']=price_path['Ex(t=2)']

In [21]:
price_path['CF_t=3']=0
price_path.loc[price_path['CF_t=2']==0.0, 'CF_t=3']=np.maximum(0.0, price_path.loc[price_path['CF_t=2']==0.0, 'Ex(t=3)'])

In [22]:
price_path.loc[price_path['ITM_t1'],'Y_reg_t1']=price_path.loc[price_path['ITM_t1'],'CF_t=2']*disc

In [23]:
N_basis=2
x=price_path.loc[price_path['ITM_t1'],'t=1'].values
y=price_path.loc[price_path['ITM_t1'],'Y_reg_t1'].values
reg = np.poly1d(np.polyfit(x,y,N_basis))

In [24]:
reg

poly1d([ 1.35645006, -3.33542735,  2.03750253])

Regression of Y (discounted value of t=2 payoff to t=1) against X (t=1 asset price) and $X^2$ gives us
   $E(Y|X) = 2.038 -3.335X + 1.356X^{2}$

In [25]:
price_path['Cont(t=1)']=np.maximum(reg(price_path['t=1']),0.0)

In [26]:
price_path['Ex(t=1)']=np.maximum(1.1-price_path['t=1'],0)

In [27]:
price_path['V(t=1)']=np.maximum(price_path['Ex(t=1)'],price_path['Cont(t=1)'])

In [28]:
price_path=price_path.set_index('Path')

In [ ]:
# You can also calculate the option price this way

In [29]:
price_path['V(t=1)'].mean()*disc

0.10947316499680305

In [30]:
stopping_rule=pd.DataFrame()
stopping_rule['Path']=price_path.index
stopping_rule.set_index('Path',inplace=True)

In [31]:
stopping_rule

""
Path
1
2
3
4
5
6
7
8


In [32]:
stopping_rule['t=1']=np.where(price_path['Cont(t=1)']<price_path['Ex(t=1)'],1,0)

In [33]:
stopping_rule['t=2']=np.where((price_path['Cont(t=2)']<price_path['Ex(t=2)']) & ~(stopping_rule['t=1'])
                              ,1,0)

In [34]:
stopping_rule['t=3']=np.where((price_path['ITM_t3']) & ~(stopping_rule['t=1']) & ~(stopping_rule['t=2'])
                              ,1,0)

In [35]:
stopping_rule

,t=1,t=2,t=3
Path,,,
1,0,0,0
2,0,0,0
3,0,0,1
4,1,0,0
5,0,0,0
6,1,0,0
7,1,0,0
8,1,0,0


In [36]:
CF[['t={}'.format(i) for i in [1,2,3]]]=(1.1-price_path[['t={}'.format(i) for i in [1,2,3]]]).values * stopping_rule.values

In [37]:
CF

,Path,t=0,t=1,t=2,t=3
0,1,0,0.00,0.0,-0.00
1,2,0,-0.00,-0.0,-0.00
2,3,0,-0.00,0.0,0.07
3,4,0,0.17,0.0,0.00
4,5,0,-0.00,-0.0,-0.00
5,6,0,0.34,0.0,0.00
6,7,0,0.18,0.0,0.00
7,8,0,0.22,-0.0,-0.00


In [38]:
for i in [1,2,3]:
    CF['t={}_disc'.format(i)]=CF['t={}'.format(i)]*(disc**i)

In [39]:
CF


,Path,t=0,t=1,t=2,t=3,t=1_disc,t=2_disc,t=3_disc
0,1,0,0.00,0.0,-0.00,0.000000,0.0,-0.000000
1,2,0,-0.00,-0.0,-0.00,-0.000000,-0.0,-0.000000
2,3,0,-0.00,0.0,0.07,-0.000000,0.0,0.058468
3,4,0,0.17,0.0,0.00,0.160099,0.0,0.000000
4,5,0,-0.00,-0.0,-0.00,-0.000000,-0.0,-0.000000
5,6,0,0.34,0.0,0.00,0.320198,0.0,0.000000
6,7,0,0.18,0.0,0.00,0.169517,0.0,0.000000
7,8,0,0.22,-0.0,-0.00,0.207187,-0.0,-0.000000


### Calcuate American Option Price by averging discounted payoff across all paths

In [40]:
opt_am=CF[['t={}_disc'.format(i) for i in [1,2,3]]].sum(axis=1).mean()

In [41]:
opt_am

0.11443370880098308

### Calculate European Option Price as a comparison

In [42]:
opt_eu=price_path['Ex(t=3)'].mean()*(disc**3)

In [43]:
opt_eu

0.05637992503615492

In [33]:
import pandas as pd
import numpy as np
x=np.ones(4)
x[1]


1.0